In [ ]:
!pip install open3d

In [ ]:
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from scipy.spatial import cKDTree

In [ ]:
def convert_point_cloud_to_mesh(ply_path, output_path=None, method='poisson', depth=9, export_format='mtl',
                                n_threads=1, clean=True):
    """
    Convert point cloud to mesh using surface reconstruction

    Args:
        ply_path: Path to input PLY file
        output_path: Path to output mesh file (default: same name with method suffix)
        method: 'poisson' or 'alpha_shape'
        depth: Depth for Poisson reconstruction (higher = more detail, but slower)
        n_threads: Number of threads for Poisson reconstruction
        clean: Whether to clean the mesh (remove isolated regions)

    Returns:
        Mesh object
    """
    # Set default output path
    if output_path is None:
        output_path = ply_path.replace('.ply', f'_{method}_mesh.{export_format}')

    pcd = o3d.io.read_point_cloud(ply_path)
    print(f"Point cloud has {len(pcd.points)} points")

    # Calculate normals if they don't exist
    if not pcd.has_normals():
        print("Estimating normals...")
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        pcd.orient_normals_consistent_tangent_plane(k=30)

    # Apply surface reconstruction
    mesh = None
    if method == 'poisson':
        print(f"Performing Poisson surface reconstruction (depth={depth})...")
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=depth, n_threads=n_threads)

        # Use vertex densities to remove low-density vertices
        if clean:
            print("Cleaning mesh...")
            vertices_to_remove = densities < np.quantile(densities, 0.05)
            mesh.remove_vertices_by_mask(vertices_to_remove)

    elif method == 'alpha_shape':
        # Estimate alpha value
        print("Performing Alpha shape reconstruction...")
        # Calculate average distance to nearest neighbors
        distances = np.asarray(pcd.compute_nearest_neighbor_distance())
        alpha = 2.0 * np.mean(distances)
        print(f"Using alpha={alpha:.4f}")

        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

    elif method == 'ball_pivoting':
        print("Performing Ball pivoting reconstruction...")
        # Calculate average distance to nearest neighbors
        distances = np.asarray(pcd.compute_nearest_neighbor_distance())
        radii = [0.005, 0.01, 0.02, 0.04]  # Multiple radii for different scales
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
            pcd, o3d.utility.DoubleVector(radii))

    else:
        raise ValueError(f"Unknown method: {method}")


    if pcd.has_colors():
        print("Transferring colors from point cloud to mesh vertices...")

        pcd_tree = cKDTree(np.asarray(pcd.points))
        mesh_vertices = np.asarray(mesh.vertices)

        dists, idxs = pcd_tree.query(mesh_vertices, k=1)
        mesh_colors = np.asarray(pcd.colors)[idxs]

        mesh.vertex_colors = o3d.utility.Vector3dVector(mesh_colors)
    else:
        print("No colors found in point cloud.")


    if mesh is None or len(mesh.triangles) == 0:
        print("Failed to create mesh. Try using a different method or parameters.")
        return None

    print(f"Created mesh with {len(mesh.triangles)} triangles")

    # Post-processing
    print("Post-processing mesh...")
    mesh.compute_vertex_normals()


    # Remove floating isolated parts
    if clean:
        try:
            with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
                triangle_clusters, cluster_n_triangles, cluster_area = mesh.cluster_connected_triangles()
            triangle_clusters = np.asarray(triangle_clusters)
            cluster_n_triangles = np.asarray(cluster_n_triangles)
            cluster_area = np.asarray(cluster_area)

            # Remove small clusters
            triangles_to_remove = cluster_n_triangles[triangle_clusters] < 100
            mesh.remove_triangles_by_mask(triangles_to_remove)
            print(f"Removed {np.sum(triangles_to_remove)} triangles from small isolated clusters")
        except Exception as e:
            print(f"Warning: Failed to remove isolated clusters: {e}")

    # Save the mesh
    print(f"Saving mesh to {output_path}")
    o3d.io.write_triangle_mesh(output_path, mesh)

    # Optionally export to .obj (note: colors will be lost unless processed with external tools)
    glb_path = ply_path.replace('.ply', f'_{method}_mesh.glb')
    o3d.io.write_triangle_mesh(glb_path, mesh, write_vertex_colors=True)

    return mesh


In [ ]:
def export_for_cloudcompare(ply_path, file_name, export_format="mtl", surface_reconstruction=True,
                           clean=True, method='poisson', depth=9):
    """
    Process a point cloud for optimal visualization in CloudCompare

    Args:
        ply_path: Path to input PLY file
        export_format: Format to export ('obj', 'ply', 'stl', 'off')
        surface_reconstruction: Whether to perform surface reconstruction
        clean: Whether to clean the output mesh
        method: Surface reconstruction method ('poisson', 'alpha_shape', 'ball_pivoting')
        depth: Depth for Poisson reconstruction

    Returns:
        Path to exported file
    """
    print(f"Processing {ply_path} for CloudCompare...")

    # Read point cloud
    pcd = o3d.io.read_point_cloud(ply_path)
    print(f"Point cloud has {len(pcd.points)} points and {len(pcd.colors)} colors")

    # Check if the point cloud has normals (needed for reconstruction)
    has_normals = pcd.has_normals()
    print(f"Has normals: {has_normals}")

    if not has_normals and surface_reconstruction:
        print("Computing normals...")
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        pcd.orient_normals_consistent_tangent_plane(k=30)

    # Output path
    if surface_reconstruction:
        output_path = ply_path.replace('.ply', f'_{method}_{file_name}_mesh.{export_format}')
        print(f"Will save reconstructed mesh to {output_path}")

        mesh = convert_point_cloud_to_mesh(
            ply_path,
            output_path=output_path,
            method=method,
            depth=depth,
            export_format=export_format,
            clean=clean
        )

        return output_path
    else:
        # If no reconstruction, just reformat the point cloud
        output_path = ply_path.replace('.ply', f'_processed.{export_format}')
        print(f"Will save processed point cloud to {output_path}")

        # For point clouds without reconstruction, PLY is the best format
        if export_format.lower() != 'ply':
            print(f"Warning: For point clouds, PLY format is recommended. {export_format} is better for meshes.")
            output_path = ply_path.replace('.ply', '_processed.ply')

        # Save with adjusted point size for better visibility
        o3d.io.write_point_cloud(output_path, pcd)
        print(f"Saved processed point cloud to {output_path}")

        return output_path

In [ ]:
export_for_cloudcompare("/content/pointcloud.ply", file_name= 'nerfmm',
                       export_format="obj",
                       surface_reconstruction=True,
                       method='poisson',  #Choose reconstruction method
                       depth=8)